<a href="https://colab.research.google.com/github/aryadneSC/Pratica-Python/blob/main/Python_Sql_28_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 . Conectando no sqlite no colab

In [ ]:
import sqlite3
con = sqlite3.connect('loja.db')
cursor = con.cursor()
cursor.execute("DROP TABLE IF EXISTS produto")
cursor.execute("DROP TABLE IF EXISTS pedido")



# 2. Criando as tabelas

In [ ]:
cursor.execute("""
CREATE TABLE pedido(
  id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  cliente TEXT NOT NULL,
  data_pedido DATE NOT NULL,
  status TEXT NOT NULL
);
""")

con.commit()

In [ ]:
cursor.execute("""
CREATE TABLE produto(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
  nome TEXT NOT NULL,
  preco REAL NOT NULL,
  pedido_id INTEGER NOT NULL,
  FOREIGN KEY(pedido_id) REFERENCES pedido(id)
);
""")
con.commit()

# 3. Alter Table

In [ ]:
cursor.execute("""
ALTER TABLE pedido ADD COLUMN forma_pagamento TEXT;
""")
con.commit()

# Inserindo informações nas tabelas

In [ ]:
cursor.execute("""
INSERT INTO pedido (cliente, data_pedido, status, forma_pagamento)
VALUES ('Vitor Felipe', '2025-11-28', 'Finalizado', 'PIX')""")
con.commit()

In [ ]:
pedidos = [
    ("Luiz Felipe", '2025-10-15', 'Em processamento', 'Cartão de Crédito'),
    ('Gloria', '2025-01-15', 'Criado', 'Boleto')
]

cursor.executemany("""
INSERT INTO pedido (cliente, data_pedido, status, forma_pagamento)
VALUES (?,?,?,?)""", pedidos)
con.commit()

In [ ]:
produtos = [
    ("Notebook", 4500.00, 1),
    ("Mouse", 150.00, 1),
    ("Teclado", 250.00, 2),
    ("Monitor", 750.00, 3),
]

cursor.executemany("""
INSERT INTO produto (nome, preco, pedido_id)
VALUES (?,?,?)""", produtos)
con.commit()

In [ ]:
cursor.execute("""
UPDATE pedido SET status = 'Finalizado' WHERE id = 2
""")
con.commit()

In [ ]:
cursor.execute("""
DELETE FROM produto WHERE nome = 'Mouse'
""")
con.commit()

In [ ]:
cursor.execute("SELECT * FROM pedido")
for linha in cursor.fetchall():
  # print(linha)
  print(linha[0])

1
2
3


In [ ]:
cursor.execute("SELECT * FROM pedido")
# print(cursor.fetchall())
# print(cursor.fetchmany(1))
linha = cursor.fetchone()

print("Linha completa", dict(linha))

# print("Cliente", linha["cliente"])
# print("Pedido", linha["data_pedido"])
# print("Status", linha["status"])
# print("Pagamento", linha["forma_pagamento"])


TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [ ]:
cursor.execute("""
SELECT
pedido.cliente,
pedido.data_pedido,
pedido.status,
produto.nome,
produto.preco
FROM pedido
INNER JOIN produto ON pedido.id = produto.pedido_id
""")

for linha in cursor.fetchall():
  print(linha)

('Vitor Felipe', '2025-11-28', 'Finalizado', 'Notebook', 4500.0)
('Luiz Felipe', '2025-10-15', 'Finalizado', 'Teclado', 250.0)
('Gloria', '2025-01-15', 'Criado', 'Monitor', 750.0)
('Vitor Felipe', '2025-11-28', 'Finalizado', 'Notebook', 4500.0)
('Luiz Felipe', '2025-10-15', 'Finalizado', 'Teclado', 250.0)
('Gloria', '2025-01-15', 'Criado', 'Monitor', 750.0)


In [ ]:
cursor.execute("""
SELECT * FROM pedido WHERE status = 'Finalizado'
""")
for linha in cursor.fetchall():
  print(linha)

(1, 'Vitor Felipe', '2025-11-28', 'Finalizado', 'PIX')
(2, 'Luiz Felipe', '2025-10-15', 'Finalizado', 'Cartão de Crédito')


# SQLAlchemy -> RAW


In [ ]:
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///zoo.db")
with engine.connect() as conn:
  conn.execute(text("DROP TABLE IF EXISTS animais"))

In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
    CREATE TABLE animais(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT,
    especie TEXT,
    idade INTEGER)"""))


In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
  INSERT INTO animais (nome, especie, idade)
  VALUES ('Rex', 'Cachorro', 3)"""))




In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 3)


In [ ]:
with engine.begin() as conn:
  conn.execute(text("""
  INSERT INTO animais (nome, especie, idade)
  VALUES
  ('Luna', 'Gato', 2),
  ('Max', 'Cachorro', 4),
  ('Bella', 'Gato', 1),
  ('Simba', 'Cachorro', 3),
  ('Oliver', 'Cachorro', 2),
  ('Milo', 'Cachorro', 1),
  ('Nala', 'Cachorro', 2)
  """))

In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))
  for r in rs:
    print(r)

(1, 'Rex', 'Cachorro', 3)
(2, 'Luna', 'Gato', 2)
(3, 'Max', 'Cachorro', 4)
(4, 'Bella', 'Gato', 1)
(5, 'Simba', 'Cachorro', 3)
(6, 'Oliver', 'Cachorro', 2)
(7, 'Milo', 'Cachorro', 1)
(8, 'Nala', 'Cachorro', 2)


In [ ]:
with engine.begin() as conn:
  rs = conn.execute(text("SELECT * FROM animais"))

  print(rs.fetchone())

(1, 'Rex', 'Cachorro', 3)


# SQLAlchemy Expression Language

In [ ]:
from sqlalchemy import create_engine,  MetaData, Table, Column, Integer, String, select, insert
engine = create_engine("sqlite:///brasil.db")
metadata = MetaData()

In [ ]:
estados = Table(
    "estados", metadata,
    Column("id", Integer, primary_key=True),
    Column("nome", String),
    Column("sigla", String)
)

In [ ]:
metadata.create_all(engine)

In [ ]:
with engine.connect() as con:
  con.execute(insert(estados).values(nome="Pernambuco", sigla = "PE"))
  rs = con.execute(select(estados))
  print(rs.fetchone())



(1, 'Pernambuco', 'PE')


# ORM – Pedidos e Produtos (1 Pedido → N Produtos)

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

# Base ORM
Base = declarative_base()

In [ ]:
# Classe Pedido
class Pedido(Base):
    __tablename__ = "pedidos"

    id = Column(Integer, primary_key=True)
    cliente = Column(String)

    # Relacionamento com produtos
    produtos = relationship("Produto", back_populates="pedido")

In [ ]:
# Classe Produto
class Produto(Base):
    __tablename__ = "produtos"

    id = Column(Integer, primary_key=True)
    nome = Column(String)
    preco = Column(Integer)
    pedido_id = Column(Integer, ForeignKey("pedidos.id"))

    # Relacionamento com pedidos
    pedido = relationship("Pedido", back_populates="produtos")

In [ ]:
# Cria banco SQLite
engine = create_engine("sqlite:///vendas.db")

In [ ]:
# Cria as tabelas
Base.metadata.create_all(engine)

In [ ]:
# Cria a sessão
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:

# ===============================
# INSERT
# ===============================

# Cria pedidos
pedido1 = Pedido(cliente="Danilo")
pedido2 = Pedido(cliente="Maria")

# Cria produtos
produto1 = Produto(nome="Teclado", preco=150, pedido=pedido1)
produto2 = Produto(nome="Mouse", preco=80, pedido=pedido1)
produto3 = Produto(nome="Monitor", preco=1200, pedido=pedido2)

# Adiciona tudo à sessão
session.add_all([pedido1, pedido2, produto1, produto2, produto3])
session.commit()

In [ ]:
# Busca pedidos com seus produtos
pedidos = session.query(Pedido).all()

for pedido in pedidos:
    print(f"\nPedido {pedido.id} - Cliente: {pedido.cliente}")
    for produto in pedido.produtos:
        print(f"  Produto: {produto.nome} - R$ {produto.preco}")


Pedido 1 - Cliente: Danilo
  Produto: Teclado - R$ 150
  Produto: Mouse - R$ 80

Pedido 2 - Cliente: Maria
  Produto: Monitor - R$ 1200


In [ ]:
session.close()
